In [1]:
import pandas as pd
import openpyxl
from openpyxl import load_workbook
import numpy as np
import win32com.client
import os
# VERSION 3.00
import re
#VERSION  3.01
import datetime


""""
- ADDED 'workbook.close()'  AND 'o.Workbooks.Close()' TO MAKE SURE THAT THE MEMORY GETS REUSED 

"""

# VERSION 3.02
"""
- FIXED THE BORDER LINES IN THE EXCEL FILES
- UPDATE ON THE EXCEL TEMPLATE (V2)
"""

'\n- FIXED THE BORDER LINES IN THE EXCEL FILES\n- UPDATE ON THE EXCEL TEMPLATE (V2)\n'

In [2]:
# DATA SOURCE (WHERE YOU ARE GETTING ALL THE DATA SCORES)

                
                #header=[0,1,2],
                   # index_col=None,sheet_name = 'PSH Scoring')
#xl = pd.ExcelFile("N:\Outcomes\CoC Performance Evaluation Process\FY2019 NOFA Performance Evaluation Documents\\KC_FY2019 Priority List Workbook Final - Full.xlsx")
xl = pd.ExcelFile("N:\Outcomes\CoC Performance Evaluation Process\FY2019 NOFA Performance Evaluation Documents\\FY2019 Priority List Workbook Final - FULL.xlsx")

pd.set_option('display.max_columns', 500)
xl.sheet_names

df = xl.parse("RRH Scoring",header=[0,1,2])
           
            

In [3]:
df.head(2)

Source is APR Data filtered to only RRH Source is APR Data filtered to only RRH  \
**Only APR Grant number Needed                   **Only APR Grant number Needed   
Project #                                                           APR Grant #   
CA0358                                                          CA0358L9D001609   
CA1336                                                          CA1336L9D001602   

Source is APR Data filtered to only RRH                            \
**Only APR Grant number Needed                                      
Project #                               Consolidated FileShare ID   
CA0358                                        CA0358       LA_COP   
CA1336                                           NaN       LA_LFH   

Source is APR Data filtered to only RRH                                           \
**Only APR Grant number Needed                                                     
Project #                                                                Grantee   
CA0358                                   Los Angeles Homeless Services Authority   
CA1336                                   Los Angeles Homeless Services Authority   

Source is APR Data filtered to only RRH                                        \
**Only APR Grant number Needed                                                  
Project #                                                              Agency   
CA0358                                                         City of Pomona   
CA1336                                   LA Family Housing Corporation (LAFH)   

Source is APR Data filtered to only RRH                                         \
**Only APR Grant number Needed                                                   
Project #                                                              Project   
CA0358                                   Pomona Rapid Re-Housing Program (New)   
CA1336                                                LFH FSC Rapid Re-Housing   

Source is APR Data filtered to only RRH                                    \
**Only APR Grant number Needed                                              
Project #                                                      Population   
CA0358                                   Chronic Homeless, Mental Illness   
CA1336                                                           Families   

Source is APR Data filtered to only RRH                           Total Score  \
**Only APR Grant number Needed                             Unnamed: 8_level_1   
Project #                               Cost Effectiveness        Prop. Score   
CA0358                                         1406.409722            0.92037   
CA1336                                        13853.900000            0.90000   

Source is APR Data filtered to only RRH                     \
**Only APR Grant number Needed          Unnamed: 9_level_1   
Project #                                   Points Awarded   
CA0358                                           92.037037   
CA1336                                           90.000000   

Source is APR Data filtered to only RRH                      \
**Only APR Grant number Needed          Unnamed: 10_level_1   
Project #                                   Points Possible   
CA0358                                                  100   
CA1336                                                  100   

Source is APR Data filtered to only RRH Scoring Area 1: HMIS Participation  \
**Only APR Grant number Needed                      Participation (APR Q6)   
Project #                                                         Response   
CA0358                                                                 Yes   
CA1336                                                                 Yes   

Source is APR Data filtered to only RRH                                \
**Only APR Grant number Needed                                          
Project #                               Poin

In [4]:
#Grantee names (checking similarities)
#df['Source is APR Data filtered to only RRH','**Only APR Grant number Needed','Grantee'].value_counts()


In [4]:
def update_range(worksheet, data, cell_range=None, named_range=None):
     """
     Updates an excel worksheet with the given data.
     :param worksheet: an excel worksheet
     :param data: data used to update the worksheet cell range (list, tuple, np.ndarray, pd.Dataframe)
     :param cell_range: a string representing the cell range, e.g. 'AB12:XX23'
     :param named_range: a string representing an excel named range
     """
 
     def clean_data(data):
         if not isinstance(data, (list, tuple, np.ndarray, pd.DataFrame)):
             raise TypeError('Invalid data, data should be an array type iterable.')
 
         if not len(data):
             raise ValueError('You need to provide data to update the cells')
 
         if isinstance(data, pd.DataFrame):
             data = data.values
 
         elif isinstance(data, (list, tuple)):
             data = np.array(data)
 
         return np.hstack(data)
 
     def clean_cells(worksheet, cell_range, named_range):
         # check that we can access a cell range
         if not any((cell_range, named_range) or all((cell_range, named_range))):
             raise ValueError('`cell_range` or `named_range` should be provided.')
 
         # get the cell range
         if cell_range:
             try:
                 cells = np.hstack(worksheet[cell_range])
             except (CellCoordinatesException, AttributeError):
                 raise ValueError('The cell range provided is invalid, cell range must be in the form XX--[:YY--]')
 
         else:
             try:
                 cells = worksheet.get_named_range(named_range)
             except (NamedRangeException, TypeError):
                 raise ValueError('The current worksheet {} does not contain any named range {}.'.format(
                     worksheet.title,
                     named_range))
 
         # checking that we have cells to update, and data
         if not len(cells):
             raise ValueError('You need to provide cells to update.')
 
         return cells
 
     cells = clean_cells(worksheet, cell_range, named_range)
     data = clean_data(data)
 
     # check that the data has the same dimension as cells
     if len(cells) != data.size:
         raise ValueError('Cells({}) should have the same dimension as the data({}).'.format(len(cells), data.size))
 
     for i, cell in enumerate(cells):
         cell.value = data[i]
            
            


In [4]:
# CREATING FOLDERS BASED ON GRANTEE NAMES
# IMPORTANT : ONLY RUN THESE CELL IF YOU WOULD LIKE TO CREATE FOLDERS inside the PHA filder (WHETHER IS PRELIMINARY OR FINAL)


for index,row in df.iterrows():
    #k = row['Grantee']
    #G = df[df['FileShare ID'].str.contains(k,regex=False)]
    k = row['Source is APR Data filtered to only RRH','**Only APR Grant number Needed','APR Grant #']
    GrantN = df[df['Source is APR Data filtered to only RRH','**Only APR Grant number Needed','APR Grant #'].str.contains(k, na = False)]

    BasePath =  r'N:\\Outcomes\CoC Performance Evaluation Process\FY2019 NOFA Performance Evaluation Documents\Priority Listing Working Documents\Evaluation Reports\Final Score Reports\PHA\\' + GrantN['Source is APR Data filtered to only RRH','**Only APR Grant number Needed']['Grantee'].to_string(index=False)
    #if not os.path.exists(os.path.join(BasePath,k)):
        #os.makedirs(os.path.join(BasePath,k))
    if not os.path.exists(BasePath):
            os.makedirs(BasePath) 

    del GrantN

C:\Users\felixa\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2819: PerformanceWarning: indexing past lexsort depth may impact performance.
  raw_cell, store_history, silent, shell_futures)


In [9]:
for index,row in df.iterrows():

    #newpath = r'C:\Users\felixa\Documents\Felix\test'
    k = row['Source is APR Data filtered to only RRH','**Only APR Grant number Needed','FileShare ID']
    newpath = r'Q:\\'  + str(k) + '\\Continuum of Care NOFA Renewals\\NOFA 2019\\Renewal Evaluations\\Final Score Reports'
    if not os.path.exists(newpath):
            os.makedirs(newpath) 


In [5]:
#df['Housing First'].head(2)

In [10]:
for index,row in df.iterrows():
    
    k = row['Source is APR Data filtered to only RRH','**Only APR Grant number Needed','APR Grant #']
    GrantN = df[df['Source is APR Data filtered to only RRH','**Only APR Grant number Needed','APR Grant #'].str.contains(k, na = False)]
    
    
    """ ----------- REMOVE THIS IF YOU WISH TO UPDATE THE EXCEL FILES--------
    #Adding the Values ----------
    VALUES1  = pd.DataFrame(index=['Housing Stability', 'Increase Income Overall', 'Commitment to Policy Priorities',
                               'Spend Down', 'Data Quality', 'HMIS Participation'],
                         columns=['Points Eearned', 'Points Possible'])
    
    
    VALUES1.iloc[0] = GrantN['Performance','Housing Stability'][['Points Earned','Points Possible']].values
    VALUES1.iloc[1] = GrantN['Performance','Increase Income Overall (APR)'][['Points Earned','Points Possible']].values
    VALUES1.iloc[2] = GrantN['Housing First','SubTotal'][['Points Earned','Points Possible']].values
    VALUES1.iloc[3] = GrantN['Scoring Area 3: Spend-Down','SubTotal'][['Points Earned','Points Possible']].values
    VALUES1.iloc[4] = GrantN['Scoring Area 1: HMIS Participation','Data Quality (APR Q7)'][['Points Earned','Points Possible']].values
    VALUES1.iloc[5] = GrantN['Scoring Area 1: HMIS Participation','Participation (APR Q6)'][['Points Earned','Points Possible']].values
    
    
    
    
    VALUES1['APR Result']  =    [GrantN['Performance','Housing Stability']['Rate'][0],
                                GrantN['Performance','Increase Income Overall (APR)']['Rate'][0],
                            
                           
                            GrantN['Housing First','Unnamed: 32_level_1']['Response'][0],
                            GrantN['Scoring Area 3: Spend-Down','Spend-Down Rate (Q31a4)']['Rate'][0],
                            GrantN['Scoring Area 1: HMIS Participation','Data Quality (APR Q7)']['Rate'][0],
                            GrantN['Scoring Area 1: HMIS Participation','Participation (APR Q6)']['Response'][0]]
             
    
    
    
    VALUES1 = VALUES1[['APR Result', 'Points Eearned', 'Points Possible']]
    VALUES1
    
    #--------------------
    
    VALUES2 = pd.DataFrame(index = ['Grant Number (APR #)', 'Grantee Name', \
                               'Operating Agency', 'Project Name', 'Project Type', 'Target Population', 'FileShare ID']
                      , columns=['values'])
    
    VALUES2.iloc[0] = GrantN['Source is APR Data filtered to only RRH','**Only APR Grant number Needed']['APR Grant #'].values
    #VALUES2.iloc[1] = GrantN['Source is APR Data filtered to only PSH','**Only APR Grant number Needed']['APR Grant #'].str[0:6].values
    VALUES2.iloc[1] = GrantN['Source is APR Data filtered to only RRH','**Only APR Grant number Needed']['Grantee'].values
    VALUES2.iloc[2] = GrantN['Source is APR Data filtered to only RRH','**Only APR Grant number Needed']['Agency'].values
    VALUES2.iloc[3] = GrantN['Source is APR Data filtered to only RRH','**Only APR Grant number Needed']['Project'].values
    # STILL NEEDS TO ADD PROJECT TYPE
    VALUES2.iloc[4] = 'RRH'#THIS COULD BE A PARAMETER! FOR OUR FUTURE FUNCTION  -FOR THIS CASE WE HAVE RRH 
    VALUES2.iloc[5] = GrantN['Source is APR Data filtered to only RRH','**Only APR Grant number Needed']['Population'].values
    VALUES2.iloc[6] = GrantN['Source is APR Data filtered to only RRH','**Only APR Grant number Needed']['FileShare ID'].values
    
    VALUES2['empty1'] = np.nan
    VALUES2['empty2'] = np.nan
    #VALUES2['empty3'] = np.nan
    
    VALUES2 = VALUES2[['empty1','values','empty2']]
    
    VALUES3 = pd.DataFrame(index= ['Score Result'], columns =['Total Points Earned', 'Total Points Possible','Project Final Score'])
    VALUES3.iloc[0] = [VALUES1['Points Eearned'].sum(),VALUES1['Points Possible'].sum() ,VALUES1['Points Eearned'].sum()/VALUES1['Points Possible'].sum() ]
    VALUES3[['Total Points Earned', 'Total Points Possible']]
    
    workbook = load_workbook('N:\Outcomes\CoC Performance Evaluation Process\FY2019 NOFA Performance Evaluation Documents\Priority Listing Working Documents\Evaluation Reports\Final Score Reports\RRH\Python\Template - RRH v2.xlsx')
    
    worksheet = workbook.active
    worksheet.title = 'template'
    update_range(workbook.active, VALUES1.values, named_range='VALUES1')
    update_range(workbook.active, VALUES2.values, named_range='VALUES2')
    
    
    #total points earned
    update_range(workbook.active, VALUES3[['Total Points Earned', 'Total Points Possible']].values, named_range='VALUES3')


    #Project Final Score
    update_range(workbook.active, VALUES3['Project Final Score'].values, named_range='VALUES4')

    # create a new excel report
    workbook.save(str(k) + '.xlsx')
    workbook.close() # ADDED
    
     ----DELETE THIS LINE ONLY IF YOU WITH TO UPDATE THE EXCEL FILE--------  """ 
    
    #Turn these into PDF Files ---------
    
    o = win32com.client.Dispatch("Excel.Application")
    o.Visible = False

    wb_path =  r'N:\Outcomes\CoC Performance Evaluation Process\FY2019 NOFA Performance Evaluation Documents\Priority Listing Working Documents\Evaluation Reports\Final Score Reports\RRH\Python\\' +  str(k)  + '.xlsx'

    wb = o.Workbooks.Open(wb_path)



    ws_index_list = [1] #say you want to print these sheets

    
    # PATH TO THE FILESHARE ID-------------------------------------------
    path_to_pdf = 'Q:\\' + GrantN['Source is APR Data filtered to only RRH','**Only APR Grant number Needed']['FileShare ID'] + '\Continuum of Care NOFA Renewals\\NOFA 2019\\Renewal Evaluations\\Final Score Reports\\' + str(k) + '.pdf'
    print(path_to_pdf)
    #--------------------------------------------------------------------
    
    # PATH TO N Drive (BY PROJECT TYPE)----------------
    #path_to_pdf = 'N:\Outcomes\CoC Performance Evaluation Process\FY2019 NOFA Performance Evaluation Documents\Priority Listing Working Documents\Evaluation Reports\Final Score Reports\RRH\PDF\\' +  str(k)  + '.pdf'
    #print(path_to_pdf)
    #--------------------------------------------------
    
    #wb.WorkSheets(ws_index_list).Select() #THIS WAS COMMENTED ON 8/21/2019

    wb.ActiveSheet.ExportAsFixedFormat(0,path_to_pdf)
    
    o.Workbooks.Close() # ADDED 
d = datetime.datetime.today()
print("FINAL REPORTS FOR RRH HAVE BEEN COMPLETED! :). Date and Time :" ,d.strftime("%d-%B-%Y %H:%M:%S") )      

C:\Users\felixa\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2819: PerformanceWarning: indexing past lexsort depth may impact performance.
  raw_cell, store_history, silent, shell_futures)


CA0358    Q:\LA_COP\Continuum of Care NOFA Renewals\NOFA...
Name: FileShare ID, dtype: object
CA1336    Q:\LA_LFH\Continuum of Care NOFA Renewals\NOFA...
Name: FileShare ID, dtype: object
CA1335    Q:\LA_UBH\Continuum of Care NOFA Renewals\NOFA...
Name: FileShare ID, dtype: object
CA1487    Q:\LA_LFH\Continuum of Care NOFA Renewals\NOFA...
Name: FileShare ID, dtype: object
CA0371    Q:\LA_PAT\Continuum of Care NOFA Renewals\NOFA...
Name: FileShare ID, dtype: object
CA1524    Q:\LA_FTS\Continuum of Care NOFA Renewals\NOFA...
Name: FileShare ID, dtype: object
CA1497    Q:\LA_UBH\Continuum of Care NOFA Renewals\NOFA...
Name: FileShare ID, dtype: object
CA0422    Q:\LA_LFH\Continuum of Care NOFA Renewals\NOFA...
Name: FileShare ID, dtype: object
CA0425    Q:\LA_PAT\Continuum of Care NOFA Renewals\NOFA...
Name: FileShare ID, dtype: object
CA1496    Q:\LA_STJ\Continuum of Care NOFA Renewals\NOFA...
Name: FileShare ID, dtype: object
CA1346    Q:\LA_HIS\Continuum of Care NOFA Renewals\NOFA...


In [7]:
#df.columns

In [10]:
# CONVERTING THE PDF FILES  AND SENDING THEM TO THEIR GRANTEE NAME(UNDER DEVELOPMENT)

for index,row in df.iterrows():
    
    k = row['Source is APR Data filtered to only RRH','**Only APR Grant number Needed','APR Grant #']
    GrantN = df[df['Source is APR Data filtered to only RRH','**Only APR Grant number Needed','APR Grant #'].str.contains(k, na = False)]
 
    o = win32com.client.Dispatch("Excel.Application")
    o.Visible = False

    wb_path =  r'N:\Outcomes\CoC Performance Evaluation Process\FY2019 NOFA Performance Evaluation Documents\Priority Listing Working Documents\Evaluation Reports\Final Score Reports\RRH\Python\\' +  str(k)  + '.xlsx'

    wb = o.Workbooks.Open(wb_path)



    ws_index_list = [1] #say you want to print these sheets

    
    # PATH TO THE FILESHARE ID
    #path_to_pdf = 'Q:\\' + GrantN['Source is APR Data filtered to only RRH','**Only APR Grant number Needed']['FileShare ID'] + '\Continuum of Care NOFA Renewals\\NOFA 2019\\Renewal Evaluations\\Preliminary Score Reports\\' + str(k) + '.pdf'
    
    # PATH TO N Drive (BY PROJECT TYPE)
    #path_to_pdf = 'N:\Outcomes\CoC Performance Evaluation Process\FY2019 NOFA Performance Evaluation Documents\Priority Listing Working Documents\Evaluation Reports\Final Score Reports\RRH\PDF\\' +  str(k)  + '.pdf'
    #print(path_to_pdf)
    
     # -------PATH TO PHA (N DRIVE, BROKEN DOWN BY GRANTEE NAME)----------------------
   
    # MAKE SURE YOU HAVE CREATED ALL THE FOLDERS BEFORE SENDING, OTHERWISE YOUR EXCEL FILE MIGHT GET STUCK
    # TRYING TO FIND A DESTINATION FOLDER THAT DOES NOT EXIST
    
    path_to_pdf = r'N:\Outcomes\CoC Performance Evaluation Process\FY2019 NOFA Performance Evaluation Documents\Priority Listing Working Documents\Evaluation Reports\Final Score Reports\PHA\\' +  GrantN['Source is APR Data filtered to only RRH','**Only APR Grant number Needed']['Grantee'].to_string(index=False)  + '\\'  +  str(k)  + '.pdf'
    print(path_to_pdf)
    
  
    
    #wb.WorkSheets(ws_index_list).Select()

    wb.ActiveSheet.ExportAsFixedFormat(0,path_to_pdf)
    
    #o.Workbooks.Close() # ADDED 

N:\Outcomes\CoC Performance Evaluation Process\FY2019 NOFA Performance Evaluation Documents\Priority Listing Working Documents\Evaluation Reports\Final Score Reports\PHA\\Los Angeles Homeless Services Authority\CA0358L9D001609.pdf


com_error: (-2147352567, 'Exception occurred.', (0, None, None, None, 0, -2147024809), None)